In [1]:
import os, sys
currentFolder = os.path.abspath('')
try:
    sys.path.remove(str(currentFolder))
except ValueError: # Already removed
    pass

projectFolder = 'C:/Users/abjawad/Documents/GitHub/local-attention-model'
sys.path.append(str(projectFolder))
os.chdir(projectFolder)
print( f"current working dir{os.getcwd()}")


current working dirC:\Users\abjawad\Documents\GitHub\local-attention-model


In [2]:

import torch
from  torch.utils.data import DataLoader
import torch.nn as nn
from models.builder import EncoderDecoder as segmodel
from dataloader.cfg_defaults import get_cfg_defaults
from utils.lr_policy import WarmUpPolyLR
from utils.init_func import init_weight, group_weight
from config_cityscapes import *
import yaml
import os
from dataloader.cityscapes_dataloader import CityscapesDataset

c:\Users\abjawad\.conda\envs\pt_cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
config_path = 'C:/Users/abjawad/Documents/GitHub/local-attention-model/dataloader/cityscapes_rgbd_config.yaml'
# with open(config_path) as info:
#     info_dict = yaml.load(info, Loader=yaml.FullLoader)
cfg = get_cfg_defaults()
cfg.merge_from_file(config_path)
cfg.freeze()

data_mean = [0.291,  0.329,  0.291]
data_std = [0.190,  0.190,  0.185]

# torch.cuda.set_per_process_memory_fraction(0.3, device=0)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

In [6]:
cityscapes_train = CityscapesDataset(cfg, split='train')
train_loader = DataLoader(cityscapes_train, batch_size=4, shuffle=True, num_workers=4, drop_last=True)
print(f'total train: {len(cityscapes_train)} t_iteration:{len(train_loader)}')
cityscapes_val = CityscapesDataset(cfg, split='val')
val_loader = DataLoader(cityscapes_val, batch_size=1, shuffle=False, num_workers=4)
print(f'total val: {len(cityscapes_val)} v_iteration:{len(val_loader)}')

Using RGB input
Using RGB input
Found 2975 train images
total train: 2975 t_iteration:743
Using RGB input
Using RGB input
Found 500 val images
total val: 500 v_iteration:500


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# Create a folder named "out" in the current directory
output_folder = "out"
os.makedirs(output_folder, exist_ok=True)

data_mean = [0.291, 0.329, 0.291]
data_std = [0.190, 0.190, 0.185]

# Loop to plot and save 200 random images
for i in range(10):
    img = cityscapes_train.__getitem__(i)  # Replace with your dataset access method
    print(img['image'].shape, type(img['image']))
    # Unnormalize the image
    img = img['image'].numpy()
    img = img.transpose((1, 2, 0))
    img *= data_std
    img += data_mean

    # Plot the image (optional)
    # plt.imshow(img)
    # plt.axis('off')  # Turn off axes for visualization
    # plt.savefig(os.path.join(output_folder, f"image_{i}.png"), bbox_inches='tight', pad_inches=0)
    # plt.close()
    break
# exit()


In [ ]:
# show one image 
import matplotlib.pyplot as plt
import numpy as np


img = cityscapes_train[0]
print(img['image'].shape)
print(img['label'].shape)

# unnormalize
img = img['image'].numpy()
img = img.transpose((1,2,0))
img *= data_std
img += data_mean
# img = img.astype(np.uint8)
plt.imshow(img)
plt.show()

label = img['label']
plt.imshow(label)
plt.show()

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=config.background)
BatchNorm2d = nn.BatchNorm2d

In [ ]:
model=segmodel(cfg=config, criterion=criterion, norm_layer=BatchNorm2d)

In [ ]:
params_list = []
params_list = group_weight(params_list, model, BatchNorm2d, config.lr)        
optimizer = torch.optim.AdamW(params_list, lr=config.lr, betas=(0.9, 0.999), weight_decay=config.weight_decay)

In [ ]:
total_iteration = config.nepochs * config.niters_per_epoch
lr_policy = WarmUpPolyLR(config.lr, config.lr_power, total_iteration, config.niters_per_epoch * config.warm_up_epoch)
print(f'lr_policy:{vars(lr_policy)}')

In [ ]:
# model = nn.DataParallel(model, device_ids = config.device_ids)
# model.to(f'cuda:{model.device_ids[0]}', non_blocking=True)


In [ ]:
from torchviz import make_dot

# images shape torch.Size([4, 3, 1024, 1024])
# gts shape torch.Size([4, 1024, 1024])

# create a random ([4, 3, 1024, 1024]) tensor
images = torch.rand(4, 3, 256, 256)

# create a random ([4, 1024, 1024]) tensor
gts = torch.randint(0, 19, (4, 256, 256))

loss, out = model(images, gts)

print('loss', loss)
print('out', out.shape)

# make no scaling
# make_dot(loss, params=dict(model.named_parameters())).render("attached", format="png")



In [ ]:
for epoch in range(1, 2):
    model.train()
    optimizer.zero_grad()
    sum_loss = 0
    m_iou_batches = 0
    for idx, sample in enumerate(train_loader):
        imgs = sample['image']
        gts = sample['label']
        imgs = imgs.to(f'cuda:{model.device_ids[0]}', non_blocking=True)
        gts = gts.to(f'cuda:{model.device_ids[0]}', non_blocking=True)  

        print('images shape', imgs.shape)
        print('gts shape', gts.shape)
        # loss, out = model(imgs, gts)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        # sum_loss += loss
        # print(f'epoch:{epoch} iteration:{idx} loss:{loss}')
        break
    # print(f'epoch:{epoch}')

In [ ]:
from torchview import draw_graph

input_size = (2, 19200, 32)
# device='meta' -> no memory is consumed for visualization
model_graph = draw_graph(model, input_size=input_size, device='meta')
model_graph.visual_graph

In [ ]:
from torchsummary import summary

# Assuming you have already created and loaded your 'model'
model = segmodel(cfg=config, criterion=criterion, norm_layer=BatchNorm2d)
# Load the pretrained weights (if not already loaded)
model.load_state_dict(torch.load('C:/Users/abjawad/Documents/GitHub/local-attention-model/pretrained/mit_b2.pth'))

# Specify the input shape (e.g., (channels, height, width))
input_shape = (4, 19200, 32)  # Adjust the input shape according to your data

# Use torchsummary to display the model summary
summary(model, input_size=input_shape, device='cpu')  # 'cpu' or 'cuda' depending on your device
